#### Regras de Associação

As regras de associação representam padrões de relacionamento entre itens de uma base de dados. Um exemplo é em transações de compras, examinando o padrão de compra de consumidores para determinar que produtos tem a tendência de ser adquiridos em conjunto. Um exemplo seria em uma lanchonete, pessoas que compram um salgado, tem a tendência de comprar suco.

Citando o famoso caso de mineração de regras de associação: "Consumidores que compram fraldas, tendem a comprar cerveja". Esse caso mostra que pode haver regras de associação muito interessante que ainda estejam escondidas nos dados. 

Para encontrar essas regras utiliza-se os parâmetros `suporte` e `confiança`. O suporte é definido como o número de transações contendo um item específico dividido pelo número total de transações.
$$ suporte(item) = \frac{Transacoes\ contendo\ o\ item}{Numero\ total\ de\ transacoes} $$

A confiança é definida como a probabilidade de comprar o item B sendo que o item A foi comprado, é calculado pelo número de transações contendo A e B, dividido pelo número de transações contendo A.
$$ confianca(A \rarr B) = \frac{Numero\ de\ transacoes\ contendo\ A\ e\ B}{Numero\ de\ transacoes\ contendo\ B} $$

In [1]:
import pandas as pd
from efficient_apriori import apriori

Primeiro é carregado em uma variável dados o conjunto de transações. Apesar de não ser um arquivo do tipo `csv` e sim de um `txt` ainda é um arquivo separado por vírgulas, então, utiliza-se o método `read_csv`.

Esse arquivo representa a simulação das transações de uma pizzaria.

In [2]:
dados = pd.read_csv("data/transacoes.txt", header=None)
dados

,0,1,2
0,Cerveja,Pizza,Sorvete
1,Pizza,Sorvete,NaN
2,Cerveja,Pizza,NaN
3,Cerveja,Pizza,Sorvete
4,Cerveja,Pizza,NaN
5,Pizza,NaN,NaN


Percebe-se que o dataframe gerado contém alguns valores faltantes, os `NaN` que serão removidos quando esse dataframe for transformado em uma lista, para ser utilizada na apriori.

Utilizando um `list comprehension` para cada linha `i` do dataframe, será pegado o elemento na posição `j` e adicionado à lista em forma de string, se e somente se, esse valor for diferente de `nan`.

In [3]:
transacoes = []
for i in range(0, 6):
    transacoes.append([str(dados.values[i, j]) for j in range(0, 3) if str(dados.values[i, j]) != "nan"])
print(transacoes)


[['Cerveja', 'Pizza', 'Sorvete'], ['Pizza', 'Sorvete'], ['Cerveja', 'Pizza'], ['Cerveja', 'Pizza', 'Sorvete'], ['Cerveja', 'Pizza'], ['Pizza']]


Dessa forma cria-se uma lista contendo as transações sem os valores `nan`.

Para executar o algoritmo basta chamar o método `apriori` e passar o conjunto de dados como partâmetro, além do suporte mínimo em `min_support` e a confiança mínima em `min_confidence`.

Esse método possui dois retornos

In [4]:
itens, regras = apriori(transacoes, min_support=0.5, min_confidence=0.5)
print("Regras:")
for regra in regras:
    print(regra)

Regras:
{Pizza} -> {Cerveja} (conf: 0.667, supp: 0.667, lift: 1.000, conv: 1.000)
{Cerveja} -> {Pizza} (conf: 1.000, supp: 0.667, lift: 1.000, conv: 0.000)
{Sorvete} -> {Pizza} (conf: 1.000, supp: 0.500, lift: 1.000, conv: 0.000)
{Pizza} -> {Sorvete} (conf: 0.500, supp: 0.500, lift: 1.000, conv: 1.000)


Para um índice de confiança maior

In [5]:
itens, regras = apriori(transacoes, min_support=0.5, min_confidence=1)
print("Regras:")
for regra in regras:
    print(regra)

Regras:
{Cerveja} -> {Pizza} (conf: 1.000, supp: 0.667, lift: 1.000, conv: 0.000)
{Sorvete} -> {Pizza} (conf: 1.000, supp: 0.500, lift: 1.000, conv: 0.000)


Observa-se que para confiança em 100% é determinado que se um cliente comprar uma Cerveja é provável que ele vá comprar uma Pizza, e se ele comprar um sorvente, é provável que ele vá comprar uma pizza. O que faz sentido pois o problema está simulando as transações de uma pizzaria.

Um outro problema seria as transações de uma mercearia. Carregando o conjunto de compras `transacoes2`.

In [6]:
dados = pd.read_csv("data/transacoes2.txt", header=None)
dados

,0,1,2,3
0,Cerveja,Carvao,Pao,NaN
1,Carvao,Pao,NaN,NaN
2,Cerveja,Carvao,Maionese,NaN
3,Cerveja,Carvao,Pao,Maionese
4,Cerveja,NaN,NaN,NaN
5,Carvao,NaN,NaN,NaN


Semelhante ao outro problema, primeiro transformar o dataframe em uma lista sem os valores faltantes `nan`.

In [7]:
transacoes = []
for i in range(0, 6):
    transacoes.append([str(dados.values[i, j]) for j in range(0, 3) if str(dados.values[i, j]) != "nan"])
print(transacoes)

[['Cerveja', 'Carvao', 'Pao'], ['Carvao', 'Pao'], ['Cerveja', 'Carvao', 'Maionese'], ['Cerveja', 'Carvao', 'Pao'], ['Cerveja'], ['Carvao']]


Aplicando o método apriori com o suporte mínimo de 50% e a confiança mínima de 75%.

In [8]:
itens, regras = apriori(transacoes, min_support=0.5, min_confidence=0.75)
print("Regras:")
for regra in regras:
    print(regra)

Regras:
{Cerveja} -> {Carvao} (conf: 0.750, supp: 0.500, lift: 0.900, conv: 0.667)
{Pao} -> {Carvao} (conf: 1.000, supp: 0.500, lift: 1.200, conv: 166666666.667)


A regras que se encontram nesse intervalo são Cerveja &rarr; Carvão e Pão &rarr; Carvão, ou seja esses são os produtos que tendem a ser comprados juntos.